### This time start without sampling and test with LSTM

Without sampling usually failed bz it is unbalanced class dataset and you need to oversampling and undersampling.

Decision Tree ( av-precison = 0.258728) without sampling  
DNN-LSTM ( av-precison = 0.25872) without sampling

In [1]:
import os 
import numpy as np 
import pandas as pd

In [2]:
def preprocess_the_whole_stage(folder_path):
    # create an empty list to store the DataFrames
    dfs = []

    # loop over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            # extract the ID from the filename (assuming the filename is in the format "ID.csv")
            file_id = os.path.splitext(filename)[0]

            # read the CSV file into a DataFrame and add the ID as a new column
            df = pd.read_csv(os.path.join(folder_path, filename))
            df.insert(0, 'ID', file_id)

            # append the DataFrame to the list
            dfs.append(df)

    # concatenate all DataFrames into a single DataFrame
    full_dataset = pd.concat(dfs)
    
    # print the resulting DataFrame
    print(full_dataset.head())
    
    # Add all zero class
    condition = (full_dataset.StartHesitation == 0) & (full_dataset.Turn == 0) & (full_dataset.Walking == 0)
    condition_2 = (full_dataset.StartHesitation == 1) | (full_dataset.Turn == 1) | (full_dataset.Walking == 1)
    
    full_dataset.loc[condition, 'All_zero'] = 1
    full_dataset.loc[condition_2, 'All_zero'] = 0
    print(full_dataset.head())
    
    print("Cleaning the Dataset")
    if 'Valid' in full_dataset.columns:
        remove_col = ['ID','Time', 'Valid', 'Task']
        
    else:
        remove_col = ['ID','Time']
    print(f"The remove columns : {remove_col}")
    clean_dataset = full_dataset.drop(full_dataset[remove_col],axis=1)
    print(clean_dataset.head())
    
    # search duplication
    print(f"Search for Duplication : {clean_dataset.duplicated().sum()}")
    clean_dataset.drop_duplicates(inplace=True)
    print(f"Search for Duplication : {clean_dataset.duplicated().sum()}")
    print(clean_dataset.head())
    
    print("Checking conditon")
    condition = (clean_dataset.StartHesitation == 0) & (clean_dataset.Turn == 0 ) & (clean_dataset.Walking == 0)
    total_zero = clean_dataset[condition].shape[0]
    print(f"Total number where three class are zero: {total_zero}")
    All_zero = clean_dataset[clean_dataset.All_zero == 1].shape[0]
    print(f"Total number of All_zero class: {All_zero}")
    print(f"Is all zero and Total number zero are equal :{All_zero == total_zero}")
    a = clean_dataset[clean_dataset.StartHesitation == 1].shape[0]
    print(f"The number of Class Start Hesitation :{a}")
    b = clean_dataset[clean_dataset.Walking == 1].shape[0]
    print(f"The number of Class Walking :{b}")
    c = clean_dataset[clean_dataset.Turn == 1].shape[0]
    print(f"The number of Class Turn : {c}")
    print(f"Is the toatl number of sample equal to all Four class combine :"
         f"{clean_dataset.shape[0] == a + b + c + All_zero}")
    
    feature_col = ['AccV','AccML','AccAP']
    label_col = ['StartHesitation','Turn','Walking', 'All_zero']

    feature_dataset = clean_dataset[feature_col]
    label_dataset = clean_dataset[label_col]
    print(f"The Feature :{feature_dataset.shape}, \n"
          f"The label {label_dataset.shape}")
    
    oversampling_dataset = pd.concat([feature_dataset, label_dataset], 
                                     ignore_index= False, sort=False, axis=1)
    
    
    print(f"Because of Four Classes are imbalanced. To get high accuracy, oversampling is used")
    from imblearn.over_sampling import SMOTE
    import numpy as np
    # Instantiate the MultiLabelUnderSampler
    over_sampler = SMOTE()

    # Undersample the dataset
    X_resampled, y_resampled = over_sampler.fit_resample(feature_dataset.to_numpy(), label_dataset.to_numpy())
    
    SMOTE_features_dataset = pd.DataFrame(X_resampled, columns=feature_dataset.columns)
    
    
    SMOTE_labels_dataset = pd.DataFrame(y_resampled, columns=label_dataset.columns)
    print(f"The over sampling label shape : {SMOTE_labels_dataset.shape}")
    
    def check_all_four_class_condition(df):
        print(f"Check all four check condiiton in {df}")
        a = df[df.StartHesitation == 1].shape[0]
        b = df[df.Turn == 1].shape[0]
        c = df[df.Walking ==1].shape[0]
        d = df[df.All_zero == 1].shape[0]
        print(f"Number of Start Hesitation : {a}, \n"
              f"Number of Turn : {b}, \n"  
              f"Number of Walking : {c}, \n"
              f"Number of All_zero : {d}")
        print("Is Number of All four class is equal to total sampling :",
             df.shape[0] == a + b + c + d)
        
    check_all_four_class_condition(SMOTE_labels_dataset)
    
    oversampling_dataset = pd.concat([SMOTE_features_dataset,SMOTE_labels_dataset], 
                                     ignore_index= False, sort=False, axis=1)
    print(f"The shape of oversampling dataset is :{oversampling_dataset.shape[0]}")
    print(f"The number of duplication in dataset{oversampling_dataset.duplicated().sum()}")
    # Drop duplication
    oversampling_dataset.drop_duplicates(inplace=True)
    print(f"The shape of oversampling after remove duplication :{oversampling_dataset.shape}")
    
    # 60% Train Data, 20% Validation Data, 20% Test Data
    # 80% Set Data(60% rain Data, 20% Validation Data) , 20% Test Data
    
    from sklearn.model_selection import train_test_split
    import random 
    random_seed = 54

    set_data, test_data = train_test_split(oversampling_dataset, test_size=0.2, random_state=True)
    print(f"The set data shape : {set_data.shape}\n"
          f"The test data shape : {test_data.shape}\n"
          f"Is the dataset still in range : "
          f"{oversampling_dataset.shape[0] == set_data.shape[0] + test_data.shape[0]}")
    
    print(f"Again Search for duplicaiton : \n "
          f"Set Data :{set_data.duplicated().sum()} \n"
          f"Test Data :{test_data.duplicated().sum()}")
    
    check_all_four_class_condition(set_data)
    check_all_four_class_condition(test_data)
    print("All task are finish")
    
    return set_data,test_data

In [3]:
def preprocessing_the_dataset(df):
#     def check_skewness(df):
#     # this can check relation between each column
#         skew_limit=0.75
#         skew_value=df[df.columns].skew()
#         #print(skew_value)
#         skew_col=skew_value[abs(skew_value)>skew_limit]
#         cols=skew_col.index
#         return cols

    import random 
    random_seed = 54
    
    feature_col = ['AccV','AccML','AccAP']
    label_col = ['StartHesitation','Turn','Walking', 'All_zero']

    # make feature and label
    feature_dataset = df[feature_col]
    label_dataset = df[label_col]
    
#     # check skewness and powertransform
#     skew_columns = check_skewness(feature_dataset)
#     print(skew_columns)
    
#     print("Power Transform start")
#     from sklearn.preprocessing import PowerTransformer
#     pt=PowerTransformer(standardize=False)  
#     feature_dataset[skew_columns] = pt.fit_transform(feature_dataset[skew_columns])
    
#     print("Standardization start")
#     # Change features data to 0 and 1
#     from sklearn.preprocessing import StandardScaler
#     sc=StandardScaler()
#     feature_dataset=sc.fit_transform(feature_dataset)
    
    print("Train test split begin")
    from sklearn.model_selection import train_test_split
    train_feature, valid_feature, train_label, valid_label = train_test_split(feature_dataset, label_dataset, test_size=0.2, random_state=True)
    
    train_feature = np.array(train_feature) 
    valid_feature = np.array(valid_feature)
    train_label  = np.array(train_label)
    valid_label = np.array(valid_label)
    print("All task are finish")
    
    return train_feature, valid_feature, train_label, valid_label
    

In [4]:
# specify the folder path
defog_path = "Data/train/defog"

In [5]:
set_defog_dataset, test_defog_dataset = preprocess_the_whole_stage(defog_path)

           ID  Time      AccV     AccML     AccAP  StartHesitation  Turn  \
0  88c6d288fb     0 -0.953125  0.078125 -0.296875                0     0   
1  88c6d288fb     1 -0.953125  0.078125 -0.296875                0     0   
2  88c6d288fb     2 -0.953125  0.078125 -0.296875                0     0   
3  88c6d288fb     3 -0.953125  0.078125 -0.296875                0     0   
4  88c6d288fb     4 -0.953125  0.078125 -0.296875                0     0   

   Walking  Valid   Task  
0        0  False  False  
1        0  False  False  
2        0  False  False  
3        0  False  False  
4        0  False  False  
           ID  Time      AccV     AccML     AccAP  StartHesitation  Turn  \
0  88c6d288fb     0 -0.953125  0.078125 -0.296875                0     0   
1  88c6d288fb     1 -0.953125  0.078125 -0.296875                0     0   
2  88c6d288fb     2 -0.953125  0.078125 -0.296875                0     0   
3  88c6d288fb     3 -0.953125  0.078125 -0.296875                0     0   
4

In [6]:
train_defog_feature, valid_defog_feature, train_defog_label, valid_defog_label = preprocessing_the_dataset(set_defog_dataset)

Train test split begin
All task are finish


In [7]:
train_defog_feature.shape, train_defog_label.shape, valid_defog_feature.shape, valid_defog_label.shape, test_defog_dataset.shape

((29126464, 3), (29126464, 4), (7281616, 3), (7281616, 4), (9102020, 7))

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report

def eval_metrics(classifier, test_features, test_labels):
    
    # make prediction
    predictions   = classifier.predict(test_features)
    
    base_score   = classifier.score(test_features, test_labels)
    accuracy = accuracy_score(test_labels, predictions)
    av_precision = average_precision_score(test_labels, predictions)
    
    target_names = ['StartHesitation','Turn','Walking', 'All_zero']
    print("Classification report")
    print("---------------------","\n")
    print(classification_report(test_labels, predictions, target_names=target_names),"\n")

    print("Accuracy Measures")
    print("---------------------","\n")
    print("Base score: ", base_score)
    print("Accuracy: ", accuracy)
    print("Avarge Precision: ", av_precision)
    
    return base_score,accuracy,av_precision

In [9]:
import mlflow
import mlflow.tensorflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
Models = {                         
    "Decision Tree": DecisionTreeClassifier(),      
    "KNearest": KNeighborsClassifier(n_jobs=-1),           
    }

In [11]:
from sklearn.model_selection import cross_val_score
counter = 1
for Model_Name, classifier in Models.items(): 
    # with mlflow.start_run(nested=True):
    print(f"{counter}. {Model_Name}")

    # fit the model
    from joblib import parallel_backend

    with parallel_backend('threading', n_jobs=-1):
        classifier.fit(train_defog_feature, train_defog_label)
        
    counter = counter + 1

    # Calculate the metrics
    base_score, accuracy, av_precision = eval_metrics(classifier,
                                                    valid_defog_feature,
                                                    valid_defog_label)  

    print("________________________________________")

1. Decision Tree
Classification report
--------------------- 



/home/hanlinn/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

StartHesitation       1.00      1.00      1.00   1820000
           Turn       0.86      0.88      0.87   1818007
        Walking       0.92      0.94      0.93   1822315
       All_zero       0.86      0.81      0.83   1821294

      micro avg       0.91      0.91      0.91   7281616
      macro avg       0.91      0.91      0.91   7281616
   weighted avg       0.91      0.91      0.91   7281616
    samples avg       0.91      0.91      0.91   7281616
 

Accuracy Measures
--------------------- 

Base score:  0.909097925515435
Accuracy:  0.909097925515435
Avarge Precision:  0.8525986776415273
________________________________________
2. KNearest
Classification report
--------------------- 

                 precision    recall  f1-score   support

StartHesitation       1.00      1.00      1.00   1820000
           Turn       0.85      0.94      0.89   1818007
        Walking       0.90      0.98      0.94   1822315
       All_zer

/home/hanlinn/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Tensorflow

In [12]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from datetime import datetime

2023-05-09 19:06:02.718252: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
epoch = 5
input_shape = 3

inputs = layers.Input(shape=input_shape, name="input_layer")
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(inputs)
x = layers.LSTM(64)(x) # return vector for whole sequence
x = layers.Dense(16, activation="relu")(x) 
outputs = layers.Dense(4, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")



model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.summary()

2023-05-09 19:06:04.066101: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 19:06:04.083528: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 19:06:04.083732: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 3)]               0         
                                                                 
 lambda (Lambda)             (None, 3, 1)              0         
                                                                 
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 16)                1040      
                                                                 
 dense_1 (Dense)             (None, 4)                 68        
                                                                 
Total params: 18,004
Trainable params: 18,004
Non-trainable params: 0
_________________________________________________________________


2023-05-09 19:06:04.886097: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-09 19:06:04.887353: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-09 19:06:04.888203: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [14]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_defog_feature, train_defog_label))
train_dataset =  train_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_defog_feature, valid_defog_label))
valid_dataset = valid_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

train_dataset, valid_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))>)

In [16]:
# fit the model
start = datetime.now()
history_model = model.fit(train_dataset,
                        batch_size=512,
                        steps_per_epoch=len(train_dataset),
                        validation_data=valid_dataset,
                        validation_steps=int(len(valid_dataset)),
                        callbacks=[early_stopping, reduce_lr],
                        epochs=epoch) 
end = datetime.now()
print(f"The time taken to train the model is {end - start}")

# Evaluate model
model.evaluate(valid_dataset)
print("________________________________________")

Epoch 1/5


2023-05-09 19:06:06.819699: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [29126464,4]
	 [[{{node Placeholder/_1}}]]
2023-05-09 19:06:07.040475: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-09 19:06:07.042301: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tens

56882/56888 [============================>.] - ETA: 0s - loss: 0.8936 - accuracy: 0.6284

2023-05-09 19:08:44.017558: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [7281616,4]
	 [[{{node Placeholder/_1}}]]
2023-05-09 19:08:44.221970: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-09 19:08:44.223331: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tenso

56888/56888 [==============================] - 175s 3ms/step - loss: 0.8936 - accuracy: 0.6284 - val_loss: 0.8153 - val_accuracy: 0.6651 - lr: 0.0010
Epoch 2/5
56888/56888 [==============================] - 176s 3ms/step - loss: 0.7930 - accuracy: 0.6748 - val_loss: 0.7783 - val_accuracy: 0.6796 - lr: 0.0010
Epoch 3/5
56888/56888 [==============================] - 177s 3ms/step - loss: 0.7668 - accuracy: 0.6840 - val_loss: 0.7585 - val_accuracy: 0.6867 - lr: 0.0010
Epoch 4/5
56888/56888 [==============================] - 177s 3ms/step - loss: 0.7510 - accuracy: 0.6890 - val_loss: 0.7416 - val_accuracy: 0.6915 - lr: 0.0010
Epoch 5/5
56888/56888 [==============================] - 173s 3ms/step - loss: 0.7394 - accuracy: 0.6929 - val_loss: 0.7342 - val_accuracy: 0.6940 - lr: 0.0010
The time taken to train the model is 0:14:37.820023
14222/14222 [==============================] - 20s 1ms/step - loss: 0.7342 - accuracy: 0.6940
________________________________________


In [18]:
# Calculate the metrics
model_preds_probs   = model.predict(valid_defog_feature)
av_precision    = average_precision_score(valid_defog_label, model_preds_probs)
av_precision

    33/227551 [..............................] - ETA: 6:05    

2023-05-09 19:28:11.312683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-09 19:28:11.314110: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-09 19:28:11.315151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

227551/227551 [==============================] - 232s 1ms/step


0.7474082063674348